In [2]:
!pip install nltk -U

     |████████████████████████████████| 1.5MB 8.2MB/s 
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [3]:
from nltk.translate.bleu_score import *
from nltk.translate.meteor_score import *
import pandas as pd
import ast

In [4]:
import os
#os.system("git clone https://github.com/soerenetler/squad-question-generation.git")
#os.system("git clone https://github.com/soerenetler/spacy_target_answer.git")
#os.system("git clone https://github.com/soerenetler/crf_target_answer.git")

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
path_to_folder = "/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/quac/question_answer/"
df = pd.read_csv(path_to_folder+"test.csv")
df["answer_sentence_token"] = [ast.literal_eval(t) for t in df["answer_sentence_token"]]
df["question_token"] = [ast.literal_eval(t) for t in df["question_token"]]

In [6]:
import unicodedata
def unicode_to_ascii(s):
    s = s.lower().strip()
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')
    
df["question_token"] = [[unicode_to_ascii(t)for t in question] for question in df["question_token"]]
df["answer_sentence_token"] = [[unicode_to_ascii(t)for t in question] for question in df["answer_sentence_token"]]

In [7]:
df.head()

,Unnamed: 0,text_title,question_text,question_token,question_pos,question_tag,question_parse_tree,answer_sentence,answer_sentence_token,answer_paragraph,masked_answer_paragraph,masked_answer_sentence,answer,unanswerable
0,0,Anna Vissi,what happened in 1983?,"[what, happened, in, 1983, ?]","['PRON', 'VERB', 'ADP', 'NUM', 'PUNCT']","['WP', 'VBD', 'IN', 'CD', '.']",NaN,"In May 1983, she married Nikos Karvelas, a com...","[in, may, 1983, ,, she, married, nikos, karvel...","In May 1983, she married Nikos Karvelas, a com...","['<<Answer>>', '<<Answer>>', '<<Answer>>', '<<...","['<<Answer>>', '<<Answer>>', '<<Answer>>', '<<...","In May 1983, she married Nikos Karvelas,",False
1,1,Anna Vissi,what happened in 1983?,"[what, happened, in, 1983, ?]","['PRON', 'VERB', 'ADP', 'NUM', 'PUNCT']","['WP', 'VBD', 'IN', 'CD', '.']",NaN,"In May 1983, she married Nikos Karvelas, a com...","[in, may, 1983, ,, she, married, nikos, karvel...","In May 1983, she married Nikos Karvelas, a com...","['<<Answer>>', '<<Answer>>', '<<Answer>>', '<<...","['<<Answer>>', '<<Answer>>', '<<Answer>>', '<<...","In May 1983, she married Nikos Karvelas, a com...",False
2,2,Anna Vissi,what happened in 1983?,"[what, happened, in, 1983, ?]","['PRON', 'VERB', 'ADP', 'NUM', 'PUNCT']","['WP', 'VBD', 'IN', 'CD', '.']",NaN,"In May 1983, she married Nikos Karvelas, a com...","[in, may, 1983, ,, she, married, nikos, karvel...","In May 1983, she married Nikos Karvelas, a com...","['<<Answer>>', '<<Answer>>', '<<Answer>>', '<<...","['<<Answer>>', '<<Answer>>', '<<Answer>>', '<<...","In May 1983, she married Nikos Karvelas,",False
3,3,Anna Vissi,what happened in 1983?,"[what, happened, in, 1983, ?]","['PRON', 'VERB', 'ADP', 'NUM', 'PUNCT']","['WP', 'VBD', 'IN', 'CD', '.']",NaN,"In May 1983, she married Nikos Karvelas, a com...","[in, may, 1983, ,, she, married, nikos, karvel...","In May 1983, she married Nikos Karvelas, a com...","['<<Answer>>', '<<Answer>>', '<<Answer>>', '<<...","['<<Answer>>', '<<Answer>>', '<<Answer>>', '<<...","In May 1983, she married Nikos Karvelas, a com...",False
4,4,Anna Vissi,did they have any children?,"[did, they, have, any, children, ?]","['AUX', 'PRON', 'VERB', 'DET', 'NOUN', 'PUNCT']","['VBD', 'PRP', 'VB', 'DT', 'NNS', '.']",NaN,"In May 1983, she married Nikos Karvelas, a com...","[in, may, 1983, ,, she, married, nikos, karvel...","In May 1983, she married Nikos Karvelas, a com...","['In', 'May', '1983', ',', 'she', 'married', '...","['in', 'may', '1983', ',', 'she', 'married', '...",she gave birth to her daughter Sofia.,False


In [11]:
sentence_bleu(["what is the original meaning of the word norman ?"], "what is the name of the anglo word ?", weights=(1,0,0,0))

0.696901560658112

## Compare Question to target sentence as reference 

In [9]:
df["sentence_question_bleu"] =[sentence_bleu([question], sentence) for sentence, question in zip(df["question_token"], df["answer_sentence_token"])]
print("BLEU: {}".format(corpus_bleu([[question] for question in df["question_token"]], df["answer_sentence_token"], weights=(.25,.25,.25,.25))*100))
print("BLEU1: {}".format(corpus_bleu([[question] for question in df["question_token"]], df["answer_sentence_token"], weights=(1,0,0,0))*100))
print("BLEU2: {}".format(corpus_bleu([[question] for question in df["question_token"]], df["answer_sentence_token"], weights=(0,1,0,0))*100))
print("BLEU3: {}".format(corpus_bleu([[question] for question in df["question_token"]], df["answer_sentence_token"], weights=(0,0,1,0))*100))
print("BLEU4: {}".format(corpus_bleu([[question] for question in df["question_token"]], df["answer_sentence_token"], weights=(0,0,0,1))*100))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  :type epsilon: float
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  :type epsilon: float
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  :type epsilon: float


BLEU: 0.4857665052891127
BLEU1: 4.531134002341652
BLEU2: 0.7259994579814774
BLEU3: 0.21970441923131104
BLEU4: 0.07704212772630274


In [10]:
import plotly.express as px
px.histogram(df, x="sentence_question_bleu")

# Compare to predicted Question

In [25]:
pred_file = "/content/gdrive/MyDrive/mt-qg-data/00_models/qg_attention/quac/dataset_quac-target_length_20-input_length_40-vocab_input_45000-max_vocab_targ_28000-epochs_15-units_512-batch_64-layer_2-dropout_0.3-pretrained_True-bidirectional_True-answer_units_0-/test.txt"

In [26]:
pred_questions= []
with open(pred_file, "r") as f:
  for line in f:
    token = line.strip().split(" ")
    try:
      end_index = token.index("<end>")
    except ValueError:
      end_index = len(token)
    pred_questions.append(token[:end_index])

In [27]:
print(pred_questions[30])
print(df["question_token"][30])

['what', 'was', 'the', 'dispute', '?']
['how', 'did', 'they', 'target', 'her', 'email', '?']


In [28]:
df["sentence_question_bleu"] =[sentence_bleu([question], sentence) for sentence, question in zip(df["question_token"], pred_questions)]
print("BLEU: {}".format(corpus_bleu([[question] for question in df["question_token"]], pred_questions, weights=(.25,.25,.25,.25))*100))
print("BLEU1: {}".format(corpus_bleu([[question] for question in df["question_token"]], pred_questions, weights=(1,0,0,0))*100))
print("BLEU2: {}".format(corpus_bleu([[question] for question in df["question_token"]], pred_questions, weights=(0,1,0,0))*100))
print("BLEU3: {}".format(corpus_bleu([[question] for question in df["question_token"]], pred_questions, weights=(0,0,1,0))*100))
print("BLEU4: {}".format(corpus_bleu([[question] for question in df["question_token"]], pred_questions, weights=(0,0,0,1))*100))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning:


Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning:


Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning:


Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().



BLEU: 7.579352478136707
BLEU1: 28.5137778498567
BLEU2: 7.626484000003102
BLEU3: 4.308408801269866
BLEU4: 3.522349030419337


# BLEU divided by answer length

In [29]:
df["pred_question"] = pred_questions
df["ans_length"] = [len(a.split(" ")) for a in df["answer"]]

all_bleu_dict = {"BLEU":[],
                 "BLEU1": [],
                 "BLEU2": [],
                 "BLEU3": [],
                 "BLEU4": []}

weight_dict = {"BLEU":(.25,.25,.25,.25),
                "BLEU1": (1,0,0,0),
                "BLEU2": (0,1,0,0),
                "BLEU3": (0,0,1,0),
                "BLEU4": (0,0,0,1)}
names = ["BLEU", "BLEU1", "BLEU2", "BLEU3", "BLEU4"]

ans_lengths = [1,2,3,4,5,6,7,8]

for name in names:
  for ans_length in ans_lengths:
    tmp_df = df[df["ans_length"]==ans_length]
    all_bleu_dict[name].append(corpus_bleu([[question] for question in tmp_df["question_token"]], tmp_df["pred_question"], weights=weight_dict[name])*100)

In [30]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=6,subplot_titles=names)
fig.update_yaxes(title_text="Score", row=1, col=1)
for i, item in enumerate(all_bleu_dict.items(),1):
  key, value=item
  fig.add_trace(
      go.Scatter(x=list(range(1,9)), y=value),
      row=1, col=i
  )
  
  fig.update_xaxes(title_text="Answer Length", row=1, col=i)

fig.update_layout(height=600, width=1400, title_text="BLEU Score by Answer Length")
fig.show()

# BLEI Score by NER-tag

In [ ]:
!pip install --quiet -U spacy[cuda110,transformers,lookups]
!python -m spacy download en_core_web_sm --quiet
!python -m spacy download en_core_web_trf --quiet
!python -m spacy download en_core_web_lg --quiet
!spacy validate

     |████████████████████████████████| 12.8MB 14.6MB/s 
     |████████████████████████████████| 9.1MB 33.3MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 460kB 41.1MB/s 
     |████████████████████████████████| 624kB 36.3MB/s 
     |████████████████████████████████| 165.3MB 92kB/s 
     |████████████████████████████████| 97.3MB 156kB/s 
     |████████████████████████████████| 122kB 62.1MB/s 
     |████████████████████████████████| 2.1MB 41.5MB/s 
     |████████████████████████████████| 1.0MB 29.1MB/s 
     |████████████████████████████████| 901kB 47.0MB/s 
     |████████████████████████████████| 3.3MB 42.2MB/s 
2021-06-12 15:26:26.335510: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.7MB 15.2MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2021-06-12 15:26:34.04634

In [ ]:
import spacy
from tqdm import tqdm

nlp = spacy.load("en_core_web_trf")

df["pred_question"] = pred_questions
ans_type_ = []
for a in tqdm(nlp.pipe(list(df["answer"]))):
  ans_type_.append(set([t.ent_type_ for t in a if t.ent_type_ != ""]))

df["ans_type"] = ans_type_

20302it [09:23, 36.02it/s]


In [ ]:

all_bleu_dict = {"BLEU":[],
                 "BLEU1": [],
                 "BLEU2": [],
                 "BLEU3": [],
                 "BLEU4": []}

weight_dict = {"BLEU":(.25,.25,.25,.25),
                "BLEU1": (1,0,0,0),
                "BLEU2": (0,1,0,0),
                "BLEU3": (0,0,1,0),
                "BLEU4": (0,0,0,1)}
names = ["BLEU", "BLEU1", "BLEU2", "BLEU3", "BLEU4"]

ans_types = ["CARDINAL", "DATE", "EVENT", "FAC", "GPE", "LANGUAGE", "LAW", "LOC", "MONEY", "NORP", "ORDINAL", "ORG", "PERCENT", "PERSON", "PRODUCT", "QUANTITY", "TIME", "WORK_OF_ART"]
ans_types_ = []
for ans_type in ans_types:
  print(ans_type)
  tmp_df = df[[ans_type in type_ for type_ in df["ans_type"]]]
  print(len(tmp_df))
  if len(tmp_df) != 0:
    ans_types_.append("{} (n={})".format(ans_type, len(tmp_df)))
    for name in names:
      all_bleu_dict[name].append(corpus_bleu([[question] for question in tmp_df["question_token"]], tmp_df["pred_question"], weights=weight_dict[name])*100)

tmp_df = df[[type_ == set() for type_ in df["ans_type"]]]
ans_types_.append("{} (n={})".format("No NER", len(tmp_df)))
for name in names:
  all_bleu_dict[name].append(corpus_bleu([[question] for question in tmp_df["question_token"]], tmp_df["pred_question"], weights=weight_dict[name])*100)

CARDINAL
1812
DATE
1398
EVENT
96
FAC
191
GPE
1165


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning:


Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().



LANGUAGE
63
LAW
76
LOC
499
MONEY
148
NORP
639
ORDINAL
156
ORG
881
PERCENT
243
PERSON
2154
PRODUCT
25
QUANTITY
319
TIME
12
WORK_OF_ART
23


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning:


Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().



In [ ]:
import plotly.express as px
tuple_list = sorted(list(zip(ans_types_, all_bleu_dict["BLEU"])), key=lambda tup: tup[1], reverse=True)
fig = px.bar(x=[t[0] for t in tuple_list][:-4], y=[t[1] for t in tuple_list][:-4])
fig.update_xaxes(title="NER-tag")
fig.update_yaxes(title="BLEU Score")

In [ ]:
spacy.explain('NORP')

'Nationalities or religious or political groups'

# Interrogative phrases compared to the Dataset
installation script based on https://colab.research.google.com/github/stanfordnlp/stanza/blob/master/demo/Stanza_CoreNLP_Interface.ipynb#scrollTo=YpKwWeVkASGt


## 1. Installation

Before the installation starts, please make sure that you have Python 3 and Java installed on your computer. Since Colab already has them installed, we'll skip this procedure in this notebook.

### Installing Stanza

Installing and importing Stanza are as simple as running the following commands:

In [ ]:
# Install stanza; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanza

# Import stanza
import stanza

### Setting up Stanford CoreNLP

In order for the interface to work, the Stanford CoreNLP library has to be installed and a `CORENLP_HOME` environment variable has to be pointed to the installation location.

Here we are going to show you how to download and install the CoreNLP library on your machine, with Stanza's installation command:

In [ ]:
# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

2021-06-12 15:38:03 WARNING: Directory ./corenlp already exists. Please install CoreNLP to a new directory.


That's all for the installation! 🎉  We can now double check if the installation is successful by listing files in the CoreNLP directory. You should be able to see a number of `.jar` files by running the following command:

In [ ]:
# Import client module
from stanza.server import CoreNLPClient

In [ ]:
# Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
client = CoreNLPClient(
    annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner', 'parse'], 
    memory='4G', 
    endpoint='http://localhost:9003',
    be_quiet=True)
print(client)

# Start the background server and wait for some time
# Note that in practice this is totally optional, as by default the server will be started when the first annotation is performed
client.start()
import time; time.sleep(10)

2021-06-12 15:39:09 INFO: Writing properties to tmp file: corenlp_server-70d508c7eefa482b.props
2021-06-12 15:39:09 INFO: Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9003 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-70d508c7eefa482b.props -annotators tokenize,ssplit,pos,lemma,ner,parse -preload -outputFormat serialized


In [ ]:
from nltk.tree import Tree
def convert_parse_tree_to_nltk_tree(parse_tree):
    return Tree(parse_tree.value, [convert_parse_tree_to_nltk_tree(child) for child in parse_tree.child]) if parse_tree.child else parse_tree.value

def get_interrogative_phrase(question_token, simple=True):
  if simple:
    return question_token[0]
  else:
    text =  " ".join(question_token)
    ann = client.annotate(text)
    parseTree = ann.sentence[0].parseTree
    subtrees = list(convert_parse_tree_to_nltk_tree(parseTree).subtrees(filter=lambda x:x.label().startswith("WH")))
    if len(subtrees)>0:
        WHP_type = subtrees[0].label()
        WHP_token = " ".join(subtrees[0].leaves()).lower()
    else:
        WHP_type = "None"
        WHP_token = "None"
    return WHP_token

In [ ]:
pipe_file = "/content/gdrive/MyDrive/mt-qg-data/00_models/qg_attention/squad/dataset_squad-target_length_20-input_length_40-vocab_input_45000-max_vocab_targ_28000-epochs_15-units_512-batch_64-layer_1-dropout_0.3-pretrained_True-bidirectional_True-answer_units_256-/trf_test.txt"

In [ ]:
trf_questions= []
with open(pipe_file, "r") as f:
  for line in f:
    token = line.strip().split(" ")
    try:
      end_index = token.index("<end>")
    except ValueError:
      end_index = len(token)
    trf_questions.append(token[:end_index])

In [ ]:
pred_wh = [get_interrogative_phrase(question_token, simple=False) for question_token in pred_questions]
true_wh = [get_interrogative_phrase(question_token, simple=False) for question_token in df["question_token"]]

In [ ]:
trf_wh = [get_interrogative_phrase(question_token, simple=False) for question_token in trf_questions]

In [ ]:
most_common_labels

In [ ]:
true_counter = Counter(true_wh)
most_common = true_counter.most_common(15)
pred_counter = Counter(pred_wh)
trf_counter = Counter(trf_wh)
most_common_labels = [e[0] for e in most_common]
print(most_common_labels)
most_common_labels.reverse()
print(most_common_labels)
most_common_true_count = [true_counter[e]/len(true_wh) for e in most_common_labels]
most_common_pred_count = [pred_counter[e]/len(pred_wh) for e in most_common_labels]
most_common_trf_count = [trf_counter[e]/len(trf_wh) for e in most_common_labels]

['what', 'who', 'None', 'when', 'where', 'how', 'why', 'how many', 'in what year', 'which', 'how much', 'how long', 'that', 'what year', 'which country']
['which country', 'what year', 'that', 'how long', 'how much', 'which', 'in what year', 'how many', 'why', 'how', 'where', 'when', 'None', 'who', 'what']


In [ ]:
import plotly.graph_objects as go


fig = go.Figure(data=[
    go.Bar( orientation='h', name='Dataset Questions', y=most_common_labels, x=most_common_true_count),
    go.Bar( orientation='h',name='Generated Questions (Answer separated)', y=most_common_labels, x=most_common_pred_count),
    go.Bar( orientation='h',name='Generated Questions (Pipeline)', y=most_common_labels, x=most_common_trf_count)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.update_layout(height=800, width=800)
fig.show()

# First Token Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
from collections import Counter
import plotly.graph_objects as go

In [ ]:
def plot_confusion_matrix(y_true, y_pred):
    most_common = Counter(y_true).most_common(100)
    most_common_labels = [e[0] for e in most_common]
    my_confusion_matrix = confusion_matrix(y_true, y_pred, labels=most_common_labels, normalize="true")

    layout = {
        "title": "Confusion Matrix", 
        "xaxis": {"title": "Predicted value"}, 
        "yaxis": {"title": "Real value"}
    }

    fig = go.Figure(data=go.Heatmap(z=my_confusion_matrix[:,:],
                                    x=most_common_labels[:5],
                                    y=most_common_labels[:5],
                                    hoverongaps=False),
                    layout=layout)
    fig.show()

pred_wh = [get_interrogative_phrase(question) for question in pred_questions]
true_wh = [get_interrogative_phrase(question) for question in df["question_token"]]

In [ ]:
plot_confusion_matrix(true_wh, pred_wh)

In [ ]:
from sklearn.metrics import classification_report
most_common = Counter(true_wh).most_common(100)
most_common_labels = [e[0] for e in most_common]
classification_report(true_wh, pred_wh, labels=most_common_labels)